In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time
import pandas as pd

servico = Service(ChromeDriverManager().install())

driver = webdriver.Chrome(service=servico)


In [13]:
#driver = webdriver.Chrome(service=servico)

#acessar site
driver.get("https://www2.aneel.gov.br/aplicacoes_liferay/tarifa/")
time.sleep(2)


In [14]:
#Seleciona Categoria do Agente Todos
select_cat_agente = Select(driver.find_element(By.NAME,"CategoriaAgente"))
select_cat_agente.select_by_visible_text('Todos')
time.sleep(1)

#Seleciona o Agente
select_agente = Select(driver.find_element(By.NAME,"Agentes"))
options_agente = [x.text for x in select_agente.options] #Traz todas as opcoes
i=0
for element in options_agente:
    if element == 'ENEL CE':
        select_agente.select_by_index(i)
        break
    i+=1

time.sleep(1)
#Seleciona Tipo de Processo
select_tipo_processo = Select(driver.find_element(By.NAME,"TipoProcesso"))
select_tipo_processo.select_by_visible_text('Todos')
time.sleep(1)
#Seleciona Ano
select_ano = Select(driver.find_element(By.NAME,"Ano"))
select_ano.select_by_visible_text('Todos')

time.sleep(1)

#Clica em Procurar
driver.find_element(By.XPATH,'/html/body/table/tbody/tr[2]/td[5]/input').click()

time.sleep(1)

#Baixa o arquivo PCAT mais atualizado (estrutura tarifaria)

time.sleep(1)
driver.find_element(By.XPATH, '/html/body/div/table/tbody/tr[4]/td[7]/a/img').click()




#select_agente.selectByValue('ENEL CE')


In [43]:
#driver.find_element(By.XPATH, '/html/body/div/table/tbody/tr[4]/td[7]/a/img').click()
#driver.find_element(By.PARTIAL_LINK_TEXT, 'PCAT').click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"partial link text","selector":"PCAT"}
  (Session info: chrome=109.0.5414.120)
Stacktrace:
Backtrace:
	(No symbol) [0x009D6643]
	(No symbol) [0x0096BE21]
	(No symbol) [0x0086DA9D]
	(No symbol) [0x008A1342]
	(No symbol) [0x008A147B]
	(No symbol) [0x008D8DC2]
	(No symbol) [0x008BFDC4]
	(No symbol) [0x008D6B09]
	(No symbol) [0x008BFB76]
	(No symbol) [0x008949C1]
	(No symbol) [0x00895E5D]
	GetHandleVerifier [0x00C4A142+2497106]
	GetHandleVerifier [0x00C785D3+2686691]
	GetHandleVerifier [0x00C7BB9C+2700460]
	GetHandleVerifier [0x00A83B10+635936]
	(No symbol) [0x00974A1F]
	(No symbol) [0x0097A418]
	(No symbol) [0x0097A505]
	(No symbol) [0x0098508B]
	BaseThreadInitThunk [0x76417D69+25]
	RtlInitializeExceptionChain [0x7778BB9B+107]
	RtlClearBits [0x7778BB1F+191]


In [27]:
#Trazer todas as opções de uma drop down
select = Select(driver.find_element(By.NAME,"Agentes"))
options = [x.text for x in select.options]
options

#options = Select(1)

#options2 = options.options()
#options2

# select by visible text
#select.select_by_visible_text('Todos')

# select by value 
#select.select_by_value('1')

['Escolha um processo',
 'Todos',
 'AME',
 'Boa Vista',
 'CAIUÁ',
 'CASTRO - DIS',
 'CEA',
 'CEB',
 'CEBDIS',
 'CEDRAP',
 'CEDRI',
 'CEEE-D',
 'CEGERO',
 'CEJAMA',
 'CELESC',
 'CELESC-DIS',
 'CELETRO',
 'CELG',
 'CELG-D',
 'CELPE',
 'CEMIG-D',
 'CEMIRIM',
 'CEPRAG',
 'CERAÇÁ',
 'CERAL ANITÁPOLIS',
 'CERAL ARAPOTI',
 'CERAL ARARUAMA',
 'CERBRANORTE',
 'CERCI',
 'CERCOS',
 'CEREJ',
 'CERES',
 'CERFOX',
 'CERGAL',
 'CERGAPA',
 'CERGRAL',
 'CERILUZ',
 'CERIM',
 'CERIPA',
 'CERIS',
 'CERMC',
 'CERMC',
 'CERMISSÕES',
 'CERMOFUL',
 'CERNHE',
 'CERON',
 'CERPALO',
 'CERPRO',
 'CERR',
 'CERRP',
 'CERSAD DISTRIBUIDORA',
 'CERSUL',
 'CERTAJA',
 'CERTEL ENERGIA',
 'CERTHIL',
 'CERTREL',
 'CERVAM',
 'CETRIL',
 'CFLO',
 'CHESP',
 'CNEE',
 'COCEL',
 'CODESAM',
 'COELBA',
 'COOPERA',
 'COOPERALIANÇA',
 'COOPERCOCAL',
 'COOPERLUZ',
 'COOPERMILA',
 'COOPERNORTE',
 'COOPERSUL',
 'COOPERZEM',
 'COORSEL',
 'COPEL-DIS',
 'COPREL',
 'COSERN',
 'CPFL Jaguari',
 'CPFL Leste Paulista',
 'CPFL Mococa',
 'CPFL- P

In [22]:
options

['Escolha uma categoria',
 'Todos',
 'Permissionária de Distribuição',
 'Concessionária de Distribuição']